In [1]:
# Installing Transformers and Datasets library
!pip install transformers==4.27.0 --q
!pip install datasets --q

In [2]:
# Import Library
from sklearn.metrics import confusion_matrix, classification_report
from sklearn.model_selection import train_test_split
from transformers import AutoModelForSequenceClassification
from transformers import Trainer
from datasets import load_metric

import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd
import numpy as np


/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/__init__.py:98: UserWarning: unable to load libtensorflow_io_plugins.so: unable to open file: libtensorflow_io_plugins.so, from paths: ['/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/libtensorflow_io_plugins.so']
caused by: ['/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/libtensorflow_io_plugins.so: undefined symbol: _ZN3tsl6StatusC1EN10tensorflow5error4CodeESt17basic_string_viewIcSt11char_traitsIcEENS_14SourceLocationE']
  warnings.warn(f"unable to load libtensorflow_io_plugins.so: {e}")
/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/__init__.py:104: UserWarning: file system plugins are not loaded: unable to open file: libtensorflow_io.so, from paths: ['/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/libtensorflow_io.so']
caused by: ['/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/libtensorflow_io.so: undefined symbol: _ZTVN10tenso

In [3]:
! pip install gdown

In [4]:
!gdown 1jGWDqO2NOhKy_WaUF9p-UsmCiRVbOwYW

Downloading...
From: https://drive.google.com/uc?id=1jGWDqO2NOhKy_WaUF9p-UsmCiRVbOwYW
To: /kaggle/working/cleaned_dataset.csv
100%|█████████████████████████████████████████| 818k/818k [00:00<00:00, 115MB/s]


In [5]:
# Read dataset and select selecting aspect
df = pd.read_csv("cleaned_dataset.csv")
df.rename(columns={"text_clean": "text"}, inplace=True)
df.replace("", float("NaN"), inplace=True)
df.dropna(inplace=True)

df['label'] = df['System']
df = df[['text', 'label']]
df.sample(5)

,text,label
514,aplikasinya membantu memanage uang fitur autod...,1
1235,baguu baru sehari invest beli portofolio hari ...,1
1633,pencairannya banget udah hari keluar notifikas...,2
866,cukup membantu fiturnya mudah dipahami pemula ...,1
821,investasi sebaiknya dipikir teman trauma kena ...,2


### **Tokenize Model**

In [6]:
from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained("indobenchmark/indobert-base-p2")

In [7]:
# function to tokenize dataset
def tokenize_function(text):
    return tokenizer(text["text"], padding='max_length', max_length=256)

### **Train Model**

In [49]:
# clearing cuda memory
import torch
torch.cuda.empty_cache()

device = "cuda:0" if torch.cuda.is_available() else "cpu"
device

'cuda:0'

In [65]:
# hyperparameter configuration for training model
from transformers import TrainingArguments

training_args = TrainingArguments(
    "test_trainer", 
    per_device_train_batch_size=32,
    per_device_eval_batch_size=32,
    num_train_epochs=4,
    learning_rate=5e-5,
    logging_steps=58,
    evaluation_strategy="steps", 
)

In [66]:
metric = load_metric("accuracy")

def compute_metrics(eval_pred):
    logits, labels = eval_pred
    predictions = np.argmax(logits, axis=-1)

    return metric.compute(predictions=predictions, references=labels)

torch.cuda.empty_cache()

# K-fold Cross Validation

In [67]:
from sklearn.model_selection import KFold
from sklearn.metrics import precision_recall_fscore_support as score
from datasets import load_dataset
from sklearn.metrics import confusion_matrix

i=0
n=10
kf = KFold(n_splits=n, random_state=30, shuffle=True)

results = []
RANDOM_SEED = 32
f1_scores = []

for train_index, test_index in kf.split(df):
  
  # splitting Dataframe (dataset not included)
  df_cv_train = df.iloc[train_index]
  df_cv_test = df.iloc[test_index]

  df_cv_eval, df_cv_test = train_test_split(
    df_cv_test,
    
    random_state=RANDOM_SEED,
  )
  
  df_cv_train.to_csv("cv_train.csv", index=False)
  df_cv_test.to_csv("cv_test.csv", index=False)
  df_cv_eval.to_csv("cv_eval.csv", index=False)

  actual_label = df_cv_test['label']

  files = {
    "train": "cv_train.csv", 
    "test": "cv_test.csv",
    "eval": "cv_eval.csv",
  }

  dataset_cv = load_dataset('csv', data_files=files)

  tokenized_datasets_cv = dataset_cv.map(tokenize_function, batched=True)

  torch.cuda.empty_cache()

  cv_train_dataset = tokenized_datasets_cv["train"]
  cv_test_dataset = tokenized_datasets_cv["test"]
  cv_eval_dataset = tokenized_datasets_cv["eval"]

  cv_model = AutoModelForSequenceClassification.from_pretrained("indobenchmark/indobert-base-p2", num_labels=3)

  cv_trainer = Trainer(
    model=cv_model,
    args=training_args,
    train_dataset=cv_train_dataset,
    eval_dataset=cv_eval_dataset,
    compute_metrics=compute_metrics,
)

  #train
  training_history = cv_trainer.train()

  # evaluate
  evaluation_history_cv = cv_trainer.evaluate()
  results.append(evaluation_history_cv['eval_accuracy'])
  i+=1

  prediction = cv_trainer.predict(cv_test_dataset)
  prediction = prediction.predictions.argmax(1)

  # report = classification_report(prediction, actual_label, target_names=["Netral", "Positif", "Negatif"])
  precision,recall,fscore,support=score(actual_label,prediction,average='macro')
  f1_scores.append(fscore)

  conf_matrix = confusion_matrix(actual_label, prediction)
  print(f"Fold {i+1} Confusion Matrix:\n{conf_matrix}\n")

Extracting data files:   0%|          | 0/3 [00:00<?, ?it/s]

Dataset csv downloaded and prepared to /root/.cache/huggingface/datasets/csv/default-f3fbd4b528472e6f/0.0.0/433e0ccc46f9880962cc2b12065189766fbb2bee57a221866138fb9203c83519. Subsequent calls will reuse this data.


/opt/conda/lib/python3.10/site-packages/datasets/packaged_modules/csv/csv.py:154: FutureWarning: the 'mangle_dupe_cols' keyword is deprecated and will be removed in a future version. Please take steps to stop the use of 'mangle_dupe_cols'
  csv_file_reader = pd.read_csv(file, iterator=True, dtype=dtype, **self.config.read_csv_kwargs)
/opt/conda/lib/python3.10/site-packages/datasets/packaged_modules/csv/csv.py:154: FutureWarning: the 'mangle_dupe_cols' keyword is deprecated and will be removed in a future version. Please take steps to stop the use of 'mangle_dupe_cols'
  csv_file_reader = pd.read_csv(file, iterator=True, dtype=dtype, **self.config.read_csv_kwargs)
/opt/conda/lib/python3.10/site-packages/datasets/packaged_modules/csv/csv.py:154: FutureWarning: the 'mangle_dupe_cols' keyword is deprecated and will be removed in a future version. Please take steps to stop the use of 'mangle_dupe_cols'
  csv_file_reader = pd.read_csv(file, iterator=True, dtype=dtype, **self.config.read_csv_

  0%|          | 0/3 [00:00<?, ?it/s]

  0%|          | 0/3 [00:00<?, ?ba/s]

...

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at indobenchmark/indobert-base-p2 and are newly initialized: ['classifier.weight', 'classifier.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
/opt/conda/lib/python3.10/site-packages/transformers/optimization.py:391: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(
/opt/conda/lib/python3.10/site-packages/torch/nn/parallel/_functions.py:68: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn('Was asked to gather along dimension 0, but all '


Step,Training Loss,Validation Loss,Accuracy
58,0.649700,0.665353,0.746667
116,0.431500,0.546834,0.786667


/opt/conda/lib/python3.10/site-packages/torch/nn/parallel/_functions.py:68: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn('Was asked to gather along dimension 0, but all '
/opt/conda/lib/python3.10/site-packages/torch/nn/parallel/_functions.py:68: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn('Was asked to gather along dimension 0, but all '


/opt/conda/lib/python3.10/site-packages/torch/nn/parallel/_functions.py:68: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn('Was asked to gather along dimension 0, but all '


Fold 11 Confusion Matrix:
[[11  2  3]
 [ 3 35  1]
 [ 3  1 16]]



In [68]:
print(f1_scores)
print(f"Mean-F1: {sum(f1_scores) / len(f1_scores)}")

[0.653110944527736, 0.7585185185185184, 0.7020595913517873, 0.7247537647537646, 0.7212885154061625, 0.6921311555457897, 0.7482456140350878, 0.7059318676965735, 0.7361737677527153, 0.791919191919192]
Mean-F1: 0.7234132931507327


In [69]:
print("results",results)
print(f"Mean-Validation: {sum(results) / len(results)}")

results [0.7511111111111111, 0.7288888888888889, 0.7866666666666666, 0.7911111111111111, 0.7066666666666667, 0.7333333333333333, 0.7333333333333333, 0.7777777777777778, 0.7555555555555555, 0.7911111111111111]
Mean-Validation: 0.7555555555555555


# Unlabel Data & Save Model

In [14]:
# !gdown 1Zv7RekcojCASu_tKDcqjHJYZ5MY5e1dt
# unlabeled_data = pd.read_csv('cleaned_unlabel_dataset.csv')

In [15]:
# unlabeled_data.sample(5)

In [16]:
# cv_model.save_pretrained("model")

In [17]:
# cv_model = AutoModelForSequenceClassification.from_pretrained("model")

In [18]:
# trainer = Trainer(model=cv_model)

In [19]:
# unlabeled_data['sentiments'] = 0

In [20]:
# def predict(text):
#     tokenized = tokenizer(text, padding='max_length', max_length=256)
#     label = trainer.predict([tokenized]).predictions.argmax(1)[0]
#     if label == 0:
#         print(f'Predicted: Netral [{label}]')
#     elif label == 1:
#         print(f'Predicted: Positif [{label}]')
#     else:
#         print(f'Predicted: Negatif [{label}]')

In [21]:
# for i, row in unlabeled_data.iterrows():
#     text = row['text_clean']
#     sentiment = predict(text)
#     unlabeled_data.at[i, 'sentiments'] = sentiment

In [22]:
# !gdown 1ytl7w06dJkbHqnzGEMzi6tuvCf7OQVk3

In [23]:
# # Tokenize data using the same tokenizer as the one used in the model
# unlabeled_data_tokenized = unlabeled_data["Ulasan"].apply((lambda x: tokenizer.encode(x, add_special_tokens=True, max_length=256, truncation=True, padding='max_length')))


In [24]:
# from torch.utils.data import Dataset

# class CustomDataset(Dataset):
#     def __init__(self, data):
#         self.data = data

#     def __len__(self):
#         return len(self.data)

#     def __getitem__(self, idx):
#         return {"input_ids": torch.tensor(self.data[idx], dtype=torch.long)}
        
# unlabeled_dataset = CustomDataset(unlabeled_data_tokenized)


In [25]:
# from torch.utils.data import DataLoader

# cv_model.eval()
# predictions = []

# with torch.no_grad():
#     for batch in DataLoader(unlabeled_dataset, batch_size=16, shuffle=False):
#         inputs = batch["input_ids"].to(device)
#         outputs = cv_model(inputs)[0]
#         predictions.append(outputs.argmax(-1).tolist())

# # Concatenate all predictions into a single list
# predictions = [item for sublist in predictions for item in sublist]


In [26]:
# from collections import Counter

# # Count the number of occurrences of each label
# label_counts = Counter(predictions)

# print(label_counts)